## INSTALL

In [ ]:
!pipenv install -qqqq anomalib[full]==1.0.0
!pipenv install matplotlib
!pipenv install pytorch-lightning

In [17]:
import os
from dotenv import load_dotenv
load_dotenv()
dataset_path = os.getenv('DATASET_PATH')
print(dataset_path)

.\bottle


## DATASET

In [18]:
def get_files_count(path):
    for root, subdirs, files in os.walk(path):
        if len(files) > 0:
            print(f"{root.split('/')[-2]}/{root.split('/')[-1]}: {len(files)}")

get_files_count(dataset_path+"/train")
print()
get_files_count(dataset_path+"/test")

.\bottle/train\good: 209

.\bottle/test\broken_large: 20
.\bottle/test\broken_small: 22
.\bottle/test\contamination: 21
.\bottle/test\good: 20


## LIBRARY & CONFIG

In [19]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from pytorch_lightning import Trainer
from torchvision.transforms import ToPILImage

from anomalib.config import get_configurable_parameters
from anomalib.data import get_datamodule
from anomalib.data.utils import read_image
from anomalib.deploy import OpenVINOInferencer
from anomalib.models import get_model
from anomalib.pre_processing.transforms import Denormalize
from anomalib.utils.callbacks import LoadModelCallback, get_callbacks

In [22]:
MODEL = "patchcore"
CONFIG_PATH = "./config/patchcore_bottle_configuration_file.yaml"
CKPT_PATH = "./config/bottle.ckpt"

In [25]:
config = get_configurable_parameters(config_path=CONFIG_PATH)

In [ ]:
datamodule = get_datamodule(config)
datamodule.prepare_data()
datamodule.setup()

mvtec: 5.26GB [57:12, 1.53MB/s]                                


In [ ]:
config.optimization.export_mode = "openvino"

model = get_model(config)
callbacks = get_callbacks(config)

In [ ]:
trainer = Trainer(**config.trainer, callbacks=callbacks)

## Training

In [ ]:
trainer.fit(model=model, datamodule=datamodule)

In [ ]:
trainer.save_checkpoint("/content/drive/MyDrive/Dataset/patchcore/bottle.ckpt")

## Test

In [ ]:
test_results = trainer.test(model=model, datamodule=datamodule, ckpt_path=CKPT_PATH)